### Model for 2 players only.

In [ ]:
# 1. Start by loading all the csv files of 2 players in a dataframe
import pandas as pd
from pathlib import Path

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path structure in Google Drive
games_folder = Path("/content/drive/MyDrive/Colab Notebooks/DL2_model/games/2_games")

print("Checking folder:", games_folder)

#Load the Files

if not games_folder.exists():
    print("Error: Folder does not exist.")
else:
    csv_files = sorted(games_folder.glob("*.csv"))
    print(f"Found {len(csv_files)} CSV files")

    if len(csv_files) == 0:
        print("No csv files found in folder")
    else:
        dfs = [pd.read_csv(file) for file in csv_files]
        combined_df = pd.concat(dfs, ignore_index=True)

        print("Combined DataFrame shape:", combined_df.shape)
        display(combined_df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checking folder: /content/drive/MyDrive/Colab Notebooks/DL2_model/games/2_games
Found 2866 CSV files
Combined DataFrame shape: (166688, 403)


,game_id,num_players,turn_number,current_player,gems_board_white,gems_board_blue,gems_board_green,gems_board_red,gems_board_black,gems_board_gold,...,gem_take2_green,gem_take2_red,gem_take2_black,noble_selection,gems_removed_white,gems_removed_blue,gems_removed_green,gems_removed_red,gems_removed_black,gems_removed_gold
0,1,2,1,0,4,4,4,4,4,5,...,0.0,2.0,0.0,-1,0,0,0,0,0,0
1,1,2,1,1,4,4,4,2,4,5,...,NaN,NaN,NaN,-1,0,0,0,0,0,0
2,1,2,2,0,3,3,4,2,3,5,...,NaN,NaN,NaN,-1,0,0,0,0,0,0
3,1,2,2,1,2,2,3,2,3,5,...,NaN,NaN,NaN,-1,0,0,0,0,0,0
4,1,2,3,0,1,1,2,2,3,5,...,NaN,NaN,NaN,-1,0,0,0,0,0,0


### 1. Feature engineering

In Splendor, optimal decision-making requires understanding both the current game state and strategic relationships between resources cards, and opponents. The 285 engineered features capture three key dimensions: (1) observable game state (gems, visible cards, nobles), (2) player capabilities (resources, reductions, victory points), and (3) derived strategic signals (card affordability, proximity to nobles, relative advantages). By pre-computing these relationships and normalizing all values to [0,1], we simplify the learning task for our MLP baseline, allowing it to focus on pattern recognition rather than complex mathematical reasoning. This approach encodes domain expertise directly into the feature space.

In [ ]:
# Dictionary to store all normalized features
normalized_features = {}

#Copy of the combined dataframe
df = combined_df.copy()
features_df = pd.DataFrame()

In [ ]:
# 1. GLOBAL features for capturing temporal context and available resources on the board ==> 10 features

# 1.1 Turn number for temporal context (early/mid/late game) ==> 1 feature
normalized_features['turn_number'] = df["turn_number"] / df['turn_number'].max()

# 1.2. Gems on board for resource availability ==> 6 features
for color in ['white', 'blue', 'green', 'red', 'black', 'gold']:
    normalized_features[f'gems_board_{color}'] = df[f'gems_board_{color}'] / df[f'gems_board_{color}'].max()

# 1.3. Normalize for remaining cards on deck
# Helps to learn about the progress of the game and time the strategies ==> 3 features
normalized_features['deck_level1_remaining'] = df['deck_level1_remaining'] / df['deck_level1_remaining'].max()
normalized_features['deck_level2_remaining'] = df['deck_level2_remaining'] / df['deck_level2_remaining'].max()
normalized_features['deck_level3_remaining'] = df['deck_level3_remaining'] / df['deck_level3_remaining'].max()

In [ ]:
# 2. VISIBLE CARDS ==> 12 cards x 7 features (each color) = 84 features
# Cards on the board represent immediate purchasing opportunities

# 2.1. Normalize for victory points and cost for each color
# The maximum for vp across all card is 5
# There are 12 cards in total

for i in range(12):
  normalized_features[f'card{i}_vp'] = df[f'card{i}_vp'] / 5.0
  normalized_features[f'card{i}_level'] = df[f'card{i}_level'] / 3.0 # Level (ordinal: 1, 2, 3 → represents difficulty/tier)
  for color in ['white', 'blue', 'green', 'red', 'black']:
    normalized_features[f'card{i}_cost_{color}'] = df[f'card{i}_cost_{color}'] / 7.0

In [ ]:
#3. NOBLES ==> 5 nobles x 6 features = 30 features
# Represent long-term strategic objectives
for i in range(5):
    # VP (always 3, but normalized for consistency)
    normalized_features[f'noble{i}_vp'] = df[f'noble{i}_vp'] / 3.0

    # Requirements (reduction bonuses needed)
    for color in ['white', 'blue', 'green', 'red', 'black']:
        normalized_features[f'noble{i}_req_{color}'] = df[f'noble{i}_req_{color}'] / 4.0

In [ ]:
# 4. PLAYER STATES (2 players × 42 features = 84 features)
# Captures resources, reductions, VP, and reserved cards for both players
for player_idx in range(2):
    # 4.1. Gems (immediate purchasing power)
    for color in ['white', 'blue', 'green', 'red', 'black', 'gold']:
        normalized_features[f'player{player_idx}_gems_{color}'] = df[f'player{player_idx}_gems_{color}'] / 10.0

    # 4.2. Permanent reductions (the "engine" - permanent discounts)
    for color in ['white', 'blue', 'green', 'red', 'black']:
        normalized_features[f'player{player_idx}_reduction_{color}'] = df[f'player{player_idx}_reduction_{color}'] / 7.0

    # 4.3. Victory points (goal is 15)
    normalized_features[f'player{player_idx}_vp'] = df[f'player{player_idx}_vp'] / 15.0

    # 4.4. Position (0 or 1 - already binary)
    normalized_features[f'player{player_idx}_position'] = df[f'player{player_idx}_position']

    # 4.5. Reserved cards (hidden strategic advantage - max 3 per player)
    for reserve_idx in range(3):
        normalized_features[f'player{player_idx}_reserved{reserve_idx}_vp'] = df[f'player{player_idx}_reserved{reserve_idx}_vp'] / 5.0
        normalized_features[f'player{player_idx}_reserved{reserve_idx}_level'] = df[f'player{player_idx}_reserved{reserve_idx}_level'] / 3.0

        for color in ['white', 'blue', 'green', 'red', 'black']:
            normalized_features[f'player{player_idx}_reserved{reserve_idx}_cost_{color}'] = df[f'player{player_idx}_reserved{reserve_idx}_cost_{color}'] / 7.0

In [ ]:
# 5. DERIVED STRATEGIC FEATURES
# Pre-computed relationships that help the MLP understand strategic situations
import numpy as np

#5.1 Affordability
# Question: "Can the active player buy this card RIGHT NOW with current resources?"
# Logic: For each card, check if (gems + reductions + gold) >= card cost

# extract current_player for fast indexing
current_player = df['current_player'].values

# Loop through each of the 12 visible cards
for card_idx in range(12):

    #1.Calculate total gold needed for PLAYER 0 to buy this card
    total_gold_p0 = np.zeros(len(df))  # Start with 0 gold needed per row

    for color in ['white', 'blue', 'green', 'red', 'black']:
        # Get card cost for this color (same for all rows)
        cost = df[f'card{card_idx}_cost_{color}'].values

        # Calculate what player 0 has available = gems + permanent reductions
        available_p0 = (df[f'player0_gems_{color}'].values +
                       df[f'player0_reduction_{color}'].values)

        # Calculate : how much is missing? (0 if already enough)
        shortfall_p0 = np.maximum(0, cost - available_p0)

        # Accumulate shortfall across all 5 colors
        total_gold_p0 += shortfall_p0

    # Check if player 0 has enough gold to cover all shortfalls
    can_afford_p0 = (total_gold_p0 <= df['player0_gems_gold'].values).astype(float)

    #Same logic for the other player

    # 2. Calculate total gold needed for PLAYER 1 to buy this card
    total_gold_p1 = np.zeros(len(df))  # Start with 0 gold needed per row

    for color in ['white', 'blue', 'green', 'red', 'black']:
        # Get card cost for this color
        cost = df[f'card{card_idx}_cost_{color}'].values

        # Calculate what player 1 has available
        available_p1 = (df[f'player1_gems_{color}'].values +
                       df[f'player1_reduction_{color}'].values)

        # Calculate shortfall for player 1
        shortfall_p1 = np.maximum(0, cost - available_p1)

        # Accumulate shortfall across all 5 colors
        total_gold_p1 += shortfall_p1

    # Check if player 1 has enough gold to cover all shortfalls
    can_afford_p1 = (total_gold_p1 <= df['player1_gems_gold'].values).astype(float)


    # 3. Select the right affordability based on whose turn it is
    # For each row: if current_player=0, use can_afford_p0, else use can_afford_p1
    normalized_features[f'can_afford_card{card_idx}'] = np.where(
        current_player == 0,  # Condition: is it player 0's turn?
        can_afford_p0,        # If yes, use player 0's affordability
        can_afford_p1         # If no, use player 1's affordability
    )

In [ ]:
# 5.2 Distances to Nobles (25 features: 5 nobles × 5 colors)
# How many more reductions needed to attract each noble?

current_player = df['current_player'].values

# Loop through each of the 5 nobles
for noble_idx in range(5):

    # Loop through each color requirement
    for color in ['white', 'blue', 'green', 'red', 'black']:

        # 1.Get noble requirement for this color (same across all rows)
        required = df[f'noble{noble_idx}_req_{color}'].values

        # 2.Get reductions owned by each player
        owned_p0 = df[f'player0_reduction_{color}'].values
        owned_p1 = df[f'player1_reduction_{color}'].values

        # 3.Calculate distance for each player
        # Distance = how many more reductions needed (0 if already satisfied)
        distance_p0 = np.maximum(0, required - owned_p0)
        distance_p1 = np.maximum(0, required - owned_p1)

        # 4.Select based on whose turn it is
        # For each row: if current_player=0, use distance_p0, else use distance_p1
        distance_active = np.where(
            current_player == 0,  # Condition: is it player 0's turn?
            distance_p0,          # If yes, use player 0's distance
            distance_p1           # If no, use player 1's distance
        )

        # 5.Normalize by max requirement (4)
        normalized_features[f'distance_noble{noble_idx}_{color}'] = distance_active / 4.0

In [ ]:
# 5.3 Relative advantages
# Compare active player vs opponent on some key metrics
# Relative VP

# Pre-extract current_player as numpy array for fast indexing
current_player = df['current_player'].values

# --- F3.1: RELATIVE VICTORY POINTS ---
# Who is winning? Positive = active player ahead, Negative = opponent ahead
vp_p0 = df['player0_vp'].values
vp_p1 = df['player1_vp'].values

# Calculate difference: active player VP - opponent VP
relative_vp = np.where(
    current_player == 0,     # If player 0's turn
    vp_p0 - vp_p1,          # Player 0 - Player 1
    vp_p1 - vp_p0           # Player 1 - Player 0
)
normalized_features['relative_vp'] = relative_vp / 15.0  # Normalize by winning VP

In [ ]:
# 5.4 RELATIVE GEMS (5 colors)
# Who has more gems of each color?
for color in ['white', 'blue', 'green', 'red', 'black']:
    gems_p0 = df[f'player0_gems_{color}'].values
    gems_p1 = df[f'player1_gems_{color}'].values

    # Calculate difference: active player gems - opponent gems
    relative_gems = np.where(
        current_player == 0,     # If player 0's turn
        gems_p0 - gems_p1,      # Player 0 - Player 1
        gems_p1 - gems_p0       # Player 1 - Player 0
    )
    normalized_features[f'relative_gems_{color}'] = relative_gems / 10.0  # Normalize by max gems


In [ ]:
# 5.5 RELATIVE REDUCTIONS (5 colors) ---
# Who has a stronger "engine" (more permanent bonuses)?
for color in ['white', 'blue', 'green', 'red', 'black']:
    reduction_p0 = df[f'player0_reduction_{color}'].values
    reduction_p1 = df[f'player1_reduction_{color}'].values

    # Calculate difference: active player reductions - opponent reductions
    relative_reductions = np.where(
        current_player == 0,     # If player 0's turn
        reduction_p0 - reduction_p1,  # Player 0 - Player 1
        reduction_p1 - reduction_p0   # Player 1 - Player 0
    )
    normalized_features[f'relative_reduction_{color}'] = relative_reductions / 7.0  # Normalize by typical max


In [ ]:
# 5.6 GEM DIVERSITY
# How many different gem colors does the active player have?
# Get gems for each player and color
gems_p0_white = df['player0_gems_white'].values
gems_p0_blue = df['player0_gems_blue'].values
gems_p0_green = df['player0_gems_green'].values
gems_p0_red = df['player0_gems_red'].values
gems_p0_black = df['player0_gems_black'].values

gems_p1_white = df['player1_gems_white'].values
gems_p1_blue = df['player1_gems_blue'].values
gems_p1_green = df['player1_gems_green'].values
gems_p1_red = df['player1_gems_red'].values
gems_p1_black = df['player1_gems_black'].values

# Count how many colors each player has (binary: has gems or not)
diversity_p0 = ((gems_p0_white > 0).astype(int) +
                (gems_p0_blue > 0).astype(int) +
                (gems_p0_green > 0).astype(int) +
                (gems_p0_red > 0).astype(int) +
                (gems_p0_black > 0).astype(int))

diversity_p1 = ((gems_p1_white > 0).astype(int) +
                (gems_p1_blue > 0).astype(int) +
                (gems_p1_green > 0).astype(int) +
                (gems_p1_red > 0).astype(int) +
                (gems_p1_black > 0).astype(int))

# Select based on current player and normalize by max (5 colors)
gem_diversity = np.where(
    current_player == 0,
    diversity_p0,
    diversity_p1
) / 5.0

normalized_features['gem_diversity'] = gem_diversity

In [ ]:
# 5.7 Total gems
# How many total gems does the active player have? (approaching 10 = must buy/reserve soon)
total_gems_p0 = (gems_p0_white + gems_p0_blue + gems_p0_green +
                 gems_p0_red + gems_p0_black + df['player0_gems_gold'].values)

total_gems_p1 = (gems_p1_white + gems_p1_blue + gems_p1_green +
                 gems_p1_red + gems_p1_black + df['player1_gems_gold'].values)

# Select based on current player and normalize by max (10 gems)
total_gems = np.where(
    current_player == 0,
    total_gems_p0,
    total_gems_p1
) / 10.0

normalized_features['total_gems'] = total_gems

In [ ]:
# 5.8 Total reduction
# How powerful is the active player (more reductions = easier to buy cards)
total_reductions_p0 = (df['player0_reduction_white'].values +
                       df['player0_reduction_blue'].values +
                       df['player0_reduction_green'].values +
                       df['player0_reduction_red'].values +
                       df['player0_reduction_black'].values)

total_reductions_p1 = (df['player1_reduction_white'].values +
                       df['player1_reduction_blue'].values +
                       df['player1_reduction_green'].values +
                       df['player1_reduction_red'].values +
                       df['player1_reduction_black'].values)

# Select based on current player and normalize by theoretical max of 35
total_reductions = np.where(
    current_player == 0,
    total_reductions_p0,
    total_reductions_p1
) / 35.0

normalized_features['total_reductions'] = total_reductions

In [ ]:
#6. Concatenate all features to avoid fragmentation of dataframe
# From dictionary to dataframe

features_df = pd.DataFrame(normalized_features)

In [ ]:
features_df['total_reductions'].head()

,total_reductions
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
# Remove redundant player0_position and player1_position from the dataframe
cols_to_remove = ["player0_position", "player1_position"]
features_df = features_df.drop(columns=cols_to_remove)

In [ ]:
# Download first 57 rows as CSV to make some manual spotchecks
features_df.head(57).to_csv('first_57_rows2.csv', index=False)

from google.colab import files
files.download('first_57_rows2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>